<a href="https://colab.research.google.com/github/abosedealli/Stockprediction/blob/main/LINEAR%20REGRESSION%20long%20and%20short%20stock_dataset_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
!pip install tensorflow
import tensorflow as tf
import random

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Importing the libraries to Predict the closing price stock price 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.callbacks import History 
import seaborn as sns
import keras
from keras.callbacks import History 
from sklearn.metrics import mean_squared_error
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [3]:
sns.set_style('whitegrid')
from datetime import datetime

## Stocks to use for analysis

---

Apple dataset from inception till till 9/1/2023 in US currency except date and volume.



In [4]:
data = pd.read_csv("/content/AAPL.dataset.csv")


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10614 entries, 0 to 10613
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       10614 non-null  object 
 1   Open       10614 non-null  float64
 2   High       10614 non-null  float64
 3   Low        10614 non-null  float64
 4   Close      10614 non-null  float64
 5   Adj Close  10614 non-null  float64
 6   Volume     10614 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 580.6+ KB


In [6]:
data.set_index(pd.DatetimeIndex(data['Date']), inplace=True)

In [7]:
data

,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
1980-12-15,15/12/1980,0.122210,0.122210,0.121652,0.121652,0.094663,175884800
1980-12-16,16/12/1980,0.113281,0.113281,0.112723,0.112723,0.087715,105728000
1980-12-17,17/12/1980,0.115513,0.116071,0.115513,0.115513,0.089886,86441600
1980-12-18,18/12/1980,0.118862,0.119420,0.118862,0.118862,0.092492,73449600
1980-12-19,19/12/1980,0.126116,0.126674,0.126116,0.126116,0.098137,48630400
...,...,...,...,...,...,...,...
2023-12-01,12/01/2023,133.880005,134.259995,131.440002,133.410004,133.410004,71379600
2023-01-13,13/01/2023,132.029999,134.919998,131.660004,134.759995,134.759995,57758000
2023-01-17,17/01/2023,134.830002,137.289993,134.130005,135.940002,135.940002,63646600


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10614 entries, 1980-12-15 to 2023-01-19
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       10614 non-null  object 
 1   Open       10614 non-null  float64
 2   High       10614 non-null  float64
 3   Low        10614 non-null  float64
 4   Close      10614 non-null  float64
 5   Adj Close  10614 non-null  float64
 6   Volume     10614 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 663.4+ KB


In [10]:
apple= data.assign(**{'average': lambda df: df.loc[:,['Open','High','Low','Close']].mean(axis=1), 
                    'EMA20': lambda df: df['average'].ewm(span=20, adjust=False).mean(), 
                    'EMA5': lambda df: df['average'].ewm(span=5, adjust=False).mean(), 
                    'dist_EMA20': lambda df: (df['average'] - df['EMA20'])/df['EMA20']*100, 
                    'dist_EMA5': lambda df: (df['average'] - df['EMA5'])/df['EMA5']*100})
apple.head()

,Date,Open,High,Low,Close,Adj Close,Volume,average,EMA20,EMA5,dist_EMA20,dist_EMA5
Date,,,,,,,,,,,,
1980-12-15,15/12/1980,0.122210,0.122210,0.121652,0.121652,0.094663,175884800,0.121931,0.121931,0.121931,0.000000,0.000000
1980-12-16,16/12/1980,0.113281,0.113281,0.112723,0.112723,0.087715,105728000,0.113002,0.121081,0.118955,-6.672099,-5.004147
1980-12-17,17/12/1980,0.115513,0.116071,0.115513,0.115513,0.089886,86441600,0.115652,0.120564,0.117854,-4.073496,-1.867943
1980-12-18,18/12/1980,0.118862,0.119420,0.118862,0.118862,0.092492,73449600,0.119002,0.120415,0.118236,-1.173757,0.647040
1980-12-19,19/12/1980,0.126116,0.126674,0.126116,0.126116,0.098137,48630400,0.126256,0.120971,0.120909,4.368292,4.421510


In [11]:
apple.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'average', 'EMA20', 'EMA5', 'dist_EMA20', 'dist_EMA5'],
      dtype='object')

In [12]:
apple

,Date,Open,High,Low,Close,Adj Close,Volume,average,EMA20,EMA5,dist_EMA20,dist_EMA5
Date,,,,,,,,,,,,
1980-12-15,15/12/1980,0.122210,0.122210,0.121652,0.121652,0.094663,175884800,0.121931,0.121931,0.121931,0.000000,0.000000
1980-12-16,16/12/1980,0.113281,0.113281,0.112723,0.112723,0.087715,105728000,0.113002,0.121081,0.118955,-6.672099,-5.004147
1980-12-17,17/12/1980,0.115513,0.116071,0.115513,0.115513,0.089886,86441600,0.115652,0.120564,0.117854,-4.073496,-1.867943
1980-12-18,18/12/1980,0.118862,0.119420,0.118862,0.118862,0.092492,73449600,0.119002,0.120415,0.118236,-1.173757,0.647040
1980-12-19,19/12/1980,0.126116,0.126674,0.126116,0.126116,0.098137,48630400,0.126256,0.120971,0.120909,4.368292,4.421510
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,12/01/2023,133.880005,134.259995,131.440002,133.410004,133.410004,71379600,133.247501,132.629022,131.210056,0.466323,1.552812
2023-01-13,13/01/2023,132.029999,134.919998,131.660004,134.759995,134.759995,57758000,133.342499,132.696972,131.920870,0.486467,1.077637
2023-01-17,17/01/2023,134.830002,137.289993,134.130005,135.940002,135.940002,63646600,135.547501,132.968451,133.129747,1.939595,1.816088


In [13]:
apple_model_data=apple[['Adj Close', 'High', 'EMA20', 'Close']]
# apple.columns

In [14]:
apple_model_data.head()

,Adj Close,High,EMA20,Close
Date,,,,
1980-12-15,0.094663,0.122210,0.121931,0.121652
1980-12-16,0.087715,0.113281,0.121081,0.112723
1980-12-17,0.089886,0.116071,0.120564,0.115513
1980-12-18,0.092492,0.119420,0.120415,0.118862
1980-12-19,0.098137,0.126674,0.120971,0.126116


In [15]:
X = apple_model_data[['Adj Close', 'EMA20', 'High']]
Y = apple_model_data['Close']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state = 99)


In [17]:
X_train.head()

,Adj Close,EMA20,High
Date,,,
1993-01-07,0.279736,0.386607,0.354911
2015-04-14,28.534246,31.499540,31.822500
2002-10-31,0.244613,0.266786,0.293571
2008-03-06,5.643310,6.522110,6.721429
1999-02-25,0.281127,0.342935,0.336496


In [18]:
print(X_test.describe())

         Adj Close        EMA20         High
count  2123.000000  2123.000000  2123.000000
mean     15.232851    15.807931    16.073653
std      34.210666    34.445820    34.970078
min       0.038213     0.056281     0.049665
25%       0.241673     0.292667     0.300223
50%       0.404594     0.481845     0.495536
75%      13.179198    15.206990    15.360000
max     178.663086   174.673936   182.940002


In [19]:
print(X_train.describe())

         Adj Close        EMA20         High
count  8491.000000  8491.000000  8491.000000
mean     16.114327    16.652285    16.968724
std      35.219935    35.288804    35.943733
min       0.039515     0.055824     0.051339
25%       0.236546     0.284489     0.293828
50%       0.404036     0.482642     0.493571
75%      14.208594    16.424707    16.525893
max     180.959732   174.927371   182.880005


In [20]:
def mean_absolute_percentage_error(y_true, y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  mape = np.mean(np.abs((y_true - y_pred) / y_true))*100
  return mape

In [35]:
from sklearn.metrics import mean_absolute_error, r2_score
#from sklearn.metrics import root_mean_squared_error 
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)

# Make the prediction
y_pred = reg.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = (mean_squared_error(y_test, y_pred))**2
r2 = r2_score(y_test, y_pred)
mape= mean_absolute_percentage_error(y_test, y_pred)
print('Mean abosulte percentage error:', mape)
print('root_mean_squared_error :', rmse)
mae = mean_absolute_error(y_test, y_pred)
print('Mean Absolute Error:', mae)
print('Mean Squared Error:', mse)
print('R Square value:', r2)

Mean abosulte percentage error: 28.957433711960917
root_mean_squared_error : 0.017524087378930565
Mean Absolute Error: 0.2049526603301111
Mean Squared Error: 0.1323785759816541
R Square value: 0.9998889641139623


Predicting to check for short term

In [36]:
#Nexflix dataset from 2020 till Dec 2022

In [37]:
n_data = pd.read_csv("/content/neflix.csv")

In [38]:
n_data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2/01/2020,326.100006,329.980011,324.779999,329.809998,329.809998,4485800
1,3/01/2020,326.779999,329.859985,325.529999,325.899994,325.899994,3806900
2,6/01/2020,323.119995,336.359985,321.200012,335.829987,335.829987,5663100
3,7/01/2020,336.470001,336.700012,330.299988,330.750000,330.750000,4703200
4,8/01/2020,331.489990,342.700012,331.049988,339.260010,339.260010,7104500
...,...,...,...,...,...,...,...
751,23/12/2022,296.179993,298.459991,291.910004,294.959991,294.959991,4251100
752,27/12/2022,293.190002,293.570007,282.130005,284.170013,284.170013,5778100
753,28/12/2022,281.920013,285.190002,273.410004,276.880005,276.880005,5964400
754,29/12/2022,283.179993,295.500000,281.010010,291.119995,291.119995,9588500


In [40]:
Nexflix= n_data.assign(**{'average': lambda df: df.loc[:,['Open','High','Low','Close']].mean(axis=1), 
                    'EMA20': lambda df: df['average'].ewm(span=20, adjust=False).mean(), 
                    'EMA5': lambda df: df['average'].ewm(span=5, adjust=False).mean(), 
                    'dist_EMA20': lambda df: (df['average'] - df['EMA20'])/df['EMA20']*100, 
                    'dist_EMA5': lambda df: (df['average'] - df['EMA5'])/df['EMA5']*100})

In [41]:
Nexflix

,Date,Open,High,Low,Close,Adj Close,Volume,average,EMA20,EMA5,dist_EMA20,dist_EMA5
0,2/01/2020,326.100006,329.980011,324.779999,329.809998,329.809998,4485800,327.667504,327.667504,327.667504,0.000000,0.000000
1,3/01/2020,326.779999,329.859985,325.529999,325.899994,325.899994,3806900,327.017494,327.605598,327.450834,-0.179516,-0.132337
2,6/01/2020,323.119995,336.359985,321.200012,335.829987,335.829987,5663100,329.127495,327.750540,328.009721,0.420123,0.340775
3,7/01/2020,336.470001,336.700012,330.299988,330.750000,330.750000,4703200,333.555000,328.303346,329.858147,1.599635,1.120740
4,8/01/2020,331.489990,342.700012,331.049988,339.260010,339.260010,7104500,336.125000,329.048266,331.947098,2.150668,1.258605
...,...,...,...,...,...,...,...,...,...,...,...,...
751,23/12/2022,296.179993,298.459991,291.910004,294.959991,294.959991,4251100,295.377495,298.255081,295.042102,-0.964807,0.113676
752,27/12/2022,293.190002,293.570007,282.130005,284.170013,284.170013,5778100,288.265007,297.303646,292.783070,-3.040205,-1.543144
753,28/12/2022,281.920013,285.190002,273.410004,276.880005,276.880005,5964400,279.350006,295.593775,288.305382,-5.495302,-3.106212
754,29/12/2022,283.179993,295.500000,281.010010,291.119995,291.119995,9588500,287.702499,294.842225,288.104421,-2.421541,-0.139506


In [42]:
Nexflix_model_data=Nexflix[['Adj Close', 'High', 'EMA20']]

Y2 = Nexflix['Close']

In [43]:
X2 = Nexflix[['Adj Close','High', 'EMA20']]

In [44]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X2, Y2, test_size=0.2, random_state = 99)

In [45]:
print(Xtest.describe())
print(Xtrain.describe())

        Adj Close        High       EMA20
count  152.000000  152.000000  152.000000
mean   435.659276  442.529670  434.158037
std    134.009562  135.167110  131.536781
min    173.100006  182.759995  181.306901
25%    338.429993  342.967506  337.825008
50%    480.455001  491.524994  487.468683
75%    525.972504  533.349991  522.199071
max    691.690002  700.989990  667.736024
        Adj Close        High       EMA20
count  604.000000  604.000000  604.000000
mean   428.712566  435.627532  429.529734
std    134.491372  135.294833  131.285829
min    166.369995  172.059998  181.221575
25%    331.775002  339.727501  334.108351
50%    480.540008  486.599991  489.014658
75%    524.842514  533.509994  521.971391
max    690.309998  694.159973  667.730079


In [46]:
def mean_absolute_percentage_error(y_truth, y_Pred):
  y_truth, y_Pred = np.array(y_truth), np.array(y_Pred)
  mape = np.mean(np.abs((y_truth - y_Pred) / y_truth))*100
  return mape

In [49]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(Xtrain, Ytrain)

# Make the prediction
y_Pred = reg.predict(Xtest)

# Evaluate the model
mse = mean_squared_error(Ytest, y_Pred)
r2 = r2_score(Ytest, y_Pred)
mape= mean_absolute_percentage_error(Ytest, y_Pred)
rmse = (mean_squared_error(Ytest, y_Pred))**2
print('Mean abosulte percentage error:', mape)
mae = mean_absolute_error(Ytest, y_Pred)
print('Mean Absolute Error:', mae)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('R Square value:', r2)

Mean abosulte percentage error: 2.945598159001199e-14
Mean Absolute Error: 7.778573107268465e-14
Mean Squared Error: 1.5103613929351514e-26
Root Mean Squared Error: 2.2811915372690107e-52
R Square value: 1.0
